In [ ]:
fname = "20221022-162507.lin.csv"

try:
    from google.colab import drive

    drive.mount("/content/drive")
    fname = f"/content/drive/MyDrive/CNFE/{fname}"
except ModuleNotFoundError:
    pass

fname

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter
from matplotlib.scale import LogScale
from pathlib import Path
import shutil


WORKDIR = Path(Path(fname).stem)
if WORKDIR.is_dir():
    shutil.rmtree(WORKDIR)
WORKDIR.mkdir()

WRITER = pd.ExcelWriter(WORKDIR / Path(fname).with_suffix(".xlsx"))

In [ ]:
df = pd.read_csv(fname, index_col=0).dropna()
df["Setup"] = df["SETUP"]
df["DataReg"] = df["DATAREG"] - df["SETUP"]
df["Enc"] = df["ENCRYPT"] - df["DATAREG"]
df["KeyGen"] = df["KEYGEN"] - df["ENCRYPT"]
df["Dec"] = df["DECRYPT"] - df["KEYGEN"]
df["diff"] = df["returned"] - df["expected"]
duration_cols = ["Setup", "DataReg", "Enc", "KeyGen", "Dec"]
df[duration_cols] = df[duration_cols] / 1000000

df = df[
    [
        "ell",
        "m",
        "n",
        "t",
        "p_1",
        "p_2",
        "lam",
        "q",
        "sd",
        "Setup",
        "DataReg",
        "Enc",
        "KeyGen",
        "Dec",
        "expected",
        "returned",
        "diff",
    ]
]
df.to_excel(WRITER, sheet_name="raw_data")
df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=120)
ax.hist(df["diff"], bins=12)
ax.set_xlabel("Noise $e$")
ax.set_ylabel("Relative Frequency")
ax.yaxis.set_major_formatter(PercentFormatter(df["diff"].count()))
fig.savefig(WORKDIR / "noise_hist.pdf")

In [ ]:
df.groupby(["ell", "n"])[duration_cols].describe().to_excel(
    WRITER, sheet_name="aggregated"
)

data = df.groupby(["ell", "n"])[duration_cols].mean().reset_index()
data["ell"] = np.rint(np.log2(data["ell"])).astype(int)
data["n"] = np.rint(np.log2(data["n"])).astype(int)

data

In [ ]:
n = 5
scope = data[data["n"] == n]

fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=120)
for col, m in zip(duration_cols, "D*o^s"):
    ax.plot(scope["ell"], scope[col], label=col, marker=m)

ax.set_xlabel("Size of DataSet $\ell$ (log of 2)")
ax.set_ylabel("Running Time (ms)")
ax.set_yscale("log")
ax.legend()
fig.savefig(WORKDIR / "comp_timing.pdf")

In [ ]:
def over_security_parameter(col):
    fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=120)

    pivot = data.pivot(index="n", columns="ell", values=col)
    for col in pivot:
        ax.plot(pivot[col], label="$\\ell = 2^{{col}}$".replace("{col}", str(col)))
    ax.set_xlabel("Security Parameter $n$ (log of 2)")
    ax.set_ylabel("Running Time (ms)")
    ax.set_yscale("log")
    ax.legend()

    return pivot, fig


pivot, fig = over_security_parameter("Setup")
pivot.to_excel(WRITER, sheet_name="time_n_setup")
fig.savefig(WORKDIR / "time_n_setup.pdf")

In [ ]:
pivot, fig = over_security_parameter("Enc")
pivot.to_excel(WRITER, sheet_name="time_n_enc")
fig.savefig(WORKDIR / "time_n_enc.pdf")

In [ ]:
WRITER.close()
shutil.make_archive(Path(fname).stem, "zip", WORKDIR)